# Chapter 13: Variable Scope and Binding

# Section 13.1: Nonlocal Variables
Python 3 added a new keyword called nonlocal. The nonlocal keyword adds a scope override to the inner scope. One of the most
common examples is to create function that can increment:

In [5]:
def counter():
    num = 0
    def incrementer():
        nonlocal num
        num += 1
        return num
    return incrementer

c = counter()
print(c())
print(c())
print(c())

1
2
3


Basically nonlocal will allow you to assign to variables in an outer scope, but not a global scope. So you can't use
nonlocal in our counter function because then it would try to assign to a global scope. Give it a try and you will
quickly get a SyntaxError . Instead you must use nonlocal in a nested function.
(Note that the functionality presented here is better implemented using generators.)

# Section 13.2: Global Variables
In Python, variables inside functions are considered local if and only if they appear in the left side of an assignment
statement, or some other binding occurrence; otherwise such a binding is looked up in enclosing functions, up to
the global scope. This is true even if the assignment statement is never executed.

In [6]:
x = 'Hi'

def read_x():
    print(x) 

read_x() 

def read_y():
    print(y) 
    
read_y() # NameError: global name 'y' is not defined


Hi


NameError: name 'y' is not defined

In [10]:
def read_y():
    y = 'Hey'
    print(y)
    
read_y()

Hey


In [12]:
def read_x_local_fail():
    if False:
        x = 'Hey'
    print(x)

read_x_local_fail() 
# UnboundLocalError: local 
# variable 'x' referenced before assignment

UnboundLocalError: local variable 'x' referenced before assignment

Normally, an assignment inside a scope will shadow any outer variables of the same name:

In [14]:
x = 'Hi'

def change_local_x():
    x = 'Bye'
    print(x)
    
change_local_x() # prints Bye
print(x) # prints Hi

Bye
Hi


Declaring a name global means that, for the rest of the scope, any assignments to the name will happen at the
module's top level:

In [16]:
x = 'Hi'
def change_global_x():
    global x
    x = 'Bye'
    print(x)
    
change_global_x() # prints Bye
print(x) # prints Bye

Bye
Bye


The global keyword means that assignments will happen at the module's top level, not at the program's top level.
Other modules will still need the usual dotted access to variables within the module.

To summarize: in order to know whether a variable x is local to a function, you should read the entire function:

    1. if you've found global x , then x is a global variable
    2. If you've found nonlocal x , then x belongs to an enclosing function, and is neither local nor global
    3. If you've found x = 5 or for x in range(3) or some other binding, then x is a local variable
    4. Otherwise x belongs to some enclosing scope (function scope, global scope, or builtins)

# Section 13.3: Local Variables
If a name is bound inside a function, it is by default accessible only within the function:

In [18]:
def foo():
    a = 5
    print(a) # ok
print(a) # NameError: name 'a' is not defined

NameError: name 'a' is not defined

Control flow constructs have no impact on the scope (with the exception of except ), but accessing variable that was
not assigned yet is an error:

In [22]:
def foo():
    if True:
        a = 5
    print(a) # ok
    
b = 3
def bar():
    if False:
        b = 5
    print(b) 
    
bar()
# UnboundLocalError: local variable 'b' 
# referenced before assignment

UnboundLocalError: local variable 'b' referenced before assignment

Common binding operations are assignments, for loops, and augmented assignments such as a += 5

# Section 13.4: The del command
This command has several related yet distinct forms.

__del v__

If v is a variable, the command __del__ v removes the variable from its scope. For example:

In [24]:
x = 5
print(x) # out: 5

del x

print(x) # NameError: name 'f' is not defined

5


NameError: name 'x' is not defined

__Note that del is a binding occurrence, which means that unless explicitly stated otherwise (using nonlocal
or global ), del v will make v local to the current scope. If you intend to delete v in an outer scope, use
nonlocal v or global v in the same scope of the del v statement.__

In all the following, the intention of a command is a default behavior but is not enforced by the language. A class
might be written in a way that invalidates this intention.

del v.name

This command triggers a call to v.\__delattr__(name).

The intention is to make the attribute name unavailable. For example:

In [26]:
class A:
    pass

a = A()
a.x = 7
print(a.x) # out: 7

del a.x

print(a.x) # error: AttributeError: 'A' object has no attribute 'x'
del v[item]

7


AttributeError: 'A' object has no attribute 'x'

This command triggers a call to v.\__delitem__(item) .

The intention is that item will not belong in the mapping implemented by the object v . For example:

In [31]:
x = {'a': 1, 'b': 2}
del x['a']

print(x) # out: {'b': 2}
print(x['a']) # error: KeyError: 'a'
del v[a:b]

{'b': 2}


KeyError: 'a'

This actually calls v.\__delslice__(a, b) .

The intention is similar to the one described above, but with slices - ranges of items instead of a single item. For
example:

In [33]:
x = [0, 1, 2, 3, 4]
del x[1:3]
print(x) # out: [0, 3, 4]

[0, 3, 4]


# Section 13.5: Functions skip class scope when looking up names

Classes have a local scope during definition, but functions inside the class do not use that scope when looking up
names. Because lambdas are functions, and comprehensions are implemented using function scope, this can lead
to some surprising behavior.

In [35]:
a = 'global'
class Fred:
    a = 'class' # class scope
    b = (a for i in range(10)) # function scope
    c = [a for i in range(10)] # function scope
    d = a # class scope
    e = lambda: a # function scope
    f = lambda a=a: a # default argument uses class scope

    @staticmethod # or @classmethod, or regular instance method
    def g(): # function scope
        return a
    
print(Fred.a) # class
print(next(Fred.b)) # global
print(Fred.c[0]) # class in Python 2, global in Python 3
print(Fred.d) # class
print(Fred.e()) # global
print(Fred.f()) # class
print(Fred.g()) # global

class
global
global
class
global
class
global


Users unfamiliar with how this scope works might expect b , c , and e to print class.

__From PEP 227:__

    Names in class scope are not accessible. Names are resolved in the innermost enclosing function scope. If a class definition occurs in a chain of nested scopes, the resolution process skips class definitions.

__From Python's documentation on naming and binding:__
    
    The scope of names defined in a class block is limited to the class block; it does not extend to the code blocks of methods – this includes comprehensions and generator expressions since they are implemented using a function scope. This means that the following will fail:

In [39]:
class A:
    a = 42
    b = list(a + int(i) for i in range(10))

TypeError: can only concatenate str (not "int") to str

This example uses references from this answer by Martijn Pieters, which contains more in depth analysis of this
behavior.

# Section 13.6: Local vs Global Scope
__What are local and global scope?__

All Python variables which are accessible at some point in code are either in local scope or in global scope.

The explanation is that local scope includes all variables defined in the current function and global scope includes
variables defined outside of the current function.

In [41]:
foo = 1     # global

def func():
    bar = 2 # local
    print(foo) # prints variable foo from global scope
    print(bar) # prints variable bar from local scope

One can inspect which variables are in which scope. Built-in functions locals() and globals() return the whole
scopes as dictionaries.

In [44]:
foo = 1

def func():
    bar = 2
    print(globals().keys()) # prints all variable names in global scope
    print(locals().keys()) # prints all variable names in local scope
func()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', '_i3', 'read_x', 'read_y', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_i10', '_i11', 'read_x_local_fail', '_i12', '_i13', 'change_local_x', '_i14', '_i15', 'change_global_x', '_i16', '_i17', 'foo', '_i18', '_i19', 'b', 'bar', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', 'A', 'a', '_i26', '_i27', '_i28', '_i29', 'x', '_i30', '_i31', '_i32', '_i33', '_i34', 'Fred', '_i35', '_i36', '_i37', '_i38', '_i39', '_i40', 'func', '_i41', '_i42', '_i43', '_i44'])
dict_keys(['bar'])


What happens with name clashes?

In [46]:
foo = 1

def func():
    foo = 2

    print(foo)   # prints 2
    
    # global variable foo still exists, unchanged:
    print(globals()['foo']) # prints 1
    print(locals()['foo']) # prints 2

To modify a global variable, use keyword global :

In [51]:
foo = 1

def func():
    global foo
    foo = 2 
func()

__The scope is defined for the whole body of the function!__

What it means is that a variable will never be global for a half of the function and local afterwards, or vice-versa.

In [50]:
foo = 1
def func():
    print(foo) 
    foo = 7
    print(foo)
func()

Likewise, the opposite:

In [54]:
foo = 1
def func():
    foo = 7
    
    print(foo) # 7
    print(globals()['foo'])
    
    global foo
    print(foo)

SyntaxError: name 'foo' is used prior to global declaration (<ipython-input-54-100fcc9e38d8>, line 8)

__Functions within functions__

There may be many levels of functions nested within functions, but within any one function there is only one local
scope for that function and the global scope. There are no intermediate scopes.

In [61]:
foo = 1
def f1():
    bar = 1
    
    def f2():
        baz = 2
        # here, foo is a global variable, baz is a local variable
        # bar is not in either scope
        print(locals().keys()) # ['baz']
        print('bar' in locals()) # False
        print('bar' in globals()) # False
    
    def f3():
        baz = 3
        print(bar) # bar from f1 is referenced so it enters local scope of f3 (closure)
        print(locals().keys()) # ['bar', 'baz']
        print('bar' in locals()) # True
        print('bar' in globals()) # False
    
    def f4():
        bar = 4 # a new local bar which hides bar from local scope of f1
        baz = 4
        print(bar)
        print(locals().keys()) # ['bar', 'baz']
        print('bar' in locals()) # True
        print('bar' in globals())# False
        

__global vs nonlocal (Python 3 only)__

Both these keywords are used to gain write access to variables which are not local to the current functions.

The global keyword declares that a name should be treated as a global variable.

In [63]:
foo = 0
# global foo

def f1():
    foo = 1
    # a new foo local in f1
    
    def f2():
        foo = 2
        # a new foo local in f2
        
        def f3():
            foo = 3 # a new foo local in f3
            print(foo) # 3
            foo = 30 # modifies local foo in f3 only
        
        def f4():
            global foo
            print(foo) # 0
            foo = 100 # modifies global foo

On the other hand, nonlocal (see Nonlocal Variables ), available in Python 3, takes a local variable from an
enclosing scope into the local scope of current function.

__The nonlocal statement causes the listed identifiers to refer to previously bound variables in the nearest
enclosing scope excluding globals.__

In [65]:
def f1():
    
    def f2():
        
        foo = 2
        # a new foo local in f2
        
        def f3():
            nonlocal foo # foo from f2, which is the nearest enclosing scope
            print(foo) # 2
            foo = 20 # modifies foo from f2!

# Section 13.7: Binding Occurrence

Each of the above statements is a binding occurrence - x become bound to the object denoted by 5 . If this statement
appears inside a function, then x will be function-local by default. See the "Syntax" section for a list of binding
statements.

# End of chapter 13